In [1]:
!nvidia-smi

Sat Jan 25 16:17:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.02              Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   41C    P8              5W /  165W |     611MiB /  16380MiB |     18%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
import torch
import transformers
import peft
import datasets
import evaluate
import time
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device

device(type='cuda', index=0)

In [3]:
dataset_path = "/app/datasets/oberon/docs/bb_ru"
model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"
#model_name = 'MTSAIR/Cotype-Nano'

EPOCHS = 7
PACKED=False

In [4]:
only_model_name = model_name.split("/")[-1].replace(':', "_")
dataset_name = dataset_path.split("/")[-1]
sft_model_path = f"/app/models/{dataset_name}_{only_model_name}"

In [5]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
peft_config = peft.LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "mlp.down_proj",
        "self_attn.k_proj",
        "self_attn.o_proj",
        "mlp.up_proj",
        "self_attn.v_proj",
        "mlp.gate_proj",
        "self_attn.q_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=peft.TaskType.CAUSAL_LM
)


In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [7]:
def load_texts():
    file_names = [] 
    for subdir, dirs, files in os.walk(dataset_path):
        if "/Rsrc/" in subdir:
            continue
        for file in files:
            file_names.append(os.path.join(subdir, file))
    texts = []
    for f in file_names:
        with open(f, 'r', encoding='utf-8') as file:
            texts.append(file.read() + tokenizer.eos_token)
    return file_names,texts
names, texts = load_texts()



In [8]:
#names = [n[len(dataset_path)+1:] for n in names]
#text_dataset = datasets.Dataset.from_dict({"texts":texts,"names" : names})

#text_dataset.push_to_hub('hodza/BlackBox.Shkola.2014')

In [16]:
block_size = 512  # Максимальная длина последовательности
stride = 32      # Шаг перекрытия    
def tokenize_with_stride(texts):
    tokenized_data = tokenizer(
        texts,
        max_length=block_size,
        truncation=True,
        stride=stride,
        return_overflowing_tokens=True,
        padding="max_length",  # Дополняем до block_size
        return_tensors="pt",   # Возвращаем тензоры PyTorch
        add_special_tokens = False,
    )
    tokenized_data["labels"] = tokenized_data["input_ids"]
    return datasets.Dataset.from_dict(tokenized_data).train_test_split(test_size=0.1,shuffle=False)
    
lm_datasets = tokenize_with_stride(texts)
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 5986
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 666
    })
})

In [17]:
print(tokenizer.decode(lm_datasets["train"][3]["input_ids"], skip_special_tokens=False))

Сборка Блэкбокса для школьных уроков информатики

«Блэкбокс  практически идеальная среда для обучения программированию.»
Т.Евсикова, научный сотрудник, 
Институт математики и информатики, Вильнюс, Литва

«... школьная сборка  это блестящая работа, лучше я еще не видел. 
Для меня главное, что показан путь, каким надо двигаться в рамках моих учебных дел. 
Это очень ценно и для развивающихся учителей: они сами будут достраивать <сборку> 
в нужном направлении. Никаких специальных институтов не нужно.»
В.В.Лаптев, профессор, 
Астраханский гос. тех. университет

«... скачал вашу сборку и как в сказку попал ... 
Ввод через любое выделение в документе, дружелюбная гипернавигация, 
<ключевые> слова <можно писать> на русском, мастер нового приложения с подсказками ...»
Александр Шостак, студент,
специальность "экономическая информатика"

Основа школьной сборки  уникальная современная бесплатная и открытая среда программирования Блэкбокс (BlackBox Component Builder), воплотившая полувековой (счит

In [11]:
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map=device,quantization_config=bnb_config,)
model._hf_peft_config_loaded = True  # silence a warning from HF trainer

Loading checkpoint shards: 100%|██████████| 2/2 [00:45<00:00, 22.56s/it]


In [12]:
model = peft.get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 29,933,568 || all params: 3,115,872,256 || trainable%: 0.9607


In [13]:
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(message)s')

In [15]:
metric = evaluate.load("bleu")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

2025-01-25 16:20:52,081 Starting new HTTPS connection (1): s3.amazonaws.com:443
2025-01-25 16:20:52,725 https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/metrics/evaluate-metric/bleu/evaluate-metric/bleu.py HTTP/11" 404 0
2025-01-25 16:20:52,731 Starting new HTTPS connection (1): huggingface.co:443
2025-01-25 16:20:53,078 https://huggingface.co:443 "HEAD /spaces/evaluate-metric/bleu/resolve/v0.4.3/bleu.py HTTP/11" 404 0
2025-01-25 16:20:53,082 Starting new HTTPS connection (1): huggingface.co:443
2025-01-25 16:20:53,438 https://huggingface.co:443 "HEAD /spaces/evaluate-metric/bleu/resolve/main/bleu.py HTTP/11" 200 0
2025-01-25 16:20:53,461 Starting new HTTPS connection (1): github.com:443
2025-01-25 16:20:53,856 https://github.com:443 "HEAD /tensorflow/nmt/raw/master/nmt/scripts/bleu.py HTTP/11" 302 0
2025-01-25 16:20:53,857 Starting new HTTPS connection (1): raw.githubusercontent.com:443
2025-01-25 16:20:54,159 https://raw.githubusercontent.com:443 "HEAD /tensorflow

In [18]:
timestr = time.strftime("%Y%m%d-%H%M%S")
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = transformers.TrainingArguments(
    output_dir=f"./results/{dataset_name}_{only_model_name}/{timestr}/",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    bf16=torch.cuda.get_device_capability(torch.cuda.current_device())[0] >= 8,  
    gradient_accumulation_steps=4,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    #save_total_limit=2,
    logging_dir='./logs',
    logging_steps=32,
    warmup_steps=100, 
    #eval_steps=32, 
    #max_steps=512,
    
    #load_best_model_at_end=True,
)
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

In [19]:
trainer.train()
#trainer.evaluate()

Epoch,Training Loss,Validation Loss
1,5.272000,0.587909
2,5.084700,0.571427
3,4.613000,0.571101
4,4.506400,0.577788
5,4.435900,0.583531
6,4.036400,0.594543


2025-01-25 22:23:05,415 Resetting dropped connection: huggingface.co
2025-01-25 22:23:05,833 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/config.json HTTP/11" 200 0
2025-01-25 22:23:06,015 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/config.json HTTP/11" 200 0
2025-01-25 23:20:05,059 Resetting dropped connection: huggingface.co
2025-01-25 23:20:05,405 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/config.json HTTP/11" 200 0
2025-01-25 23:20:05,565 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/config.json HTTP/11" 200 0
2025-01-26 00:16:55,934 Resetting dropped connection: huggingface.co
2025-01-26 00:16:56,320 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/config.json HTTP/11" 200 0
2025-01-26 00:16:56,480 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/config.json HTTP/11" 200 0
2025-01-26 01:13:46

TrainOutput(global_step=5236, training_loss=4.567691858348453, metrics={'train_runtime': 23871.8997, 'train_samples_per_second': 1.755, 'train_steps_per_second': 0.219, 'total_flos': 3.605993812729528e+17, 'train_loss': 4.567691858348453, 'epoch': 6.99164717674574})

In [125]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 5.12


In [66]:
final_path = sft_model_path + f"_{timestr}"
model.save_pretrained(final_path)
tokenizer.save_pretrained(final_path)

2025-01-26 09:43:34,014 Resetting dropped connection: huggingface.co
2025-01-26 09:43:34,376 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/config.json HTTP/11" 200 0
2025-01-26 09:43:34,568 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/config.json HTTP/11" 200 0


('/app/models/bb_ru_Qwen2.5-Coder-3B-Instruct_20250125-212602/tokenizer_config.json',
 '/app/models/bb_ru_Qwen2.5-Coder-3B-Instruct_20250125-212602/special_tokens_map.json',
 '/app/models/bb_ru_Qwen2.5-Coder-3B-Instruct_20250125-212602/vocab.json',
 '/app/models/bb_ru_Qwen2.5-Coder-3B-Instruct_20250125-212602/merges.txt',
 '/app/models/bb_ru_Qwen2.5-Coder-3B-Instruct_20250125-212602/added_tokens.json',
 '/app/models/bb_ru_Qwen2.5-Coder-3B-Instruct_20250125-212602/tokenizer.json')

In [67]:
final_path

'/app/models/bb_ru_Qwen2.5-Coder-3B-Instruct_20250125-212602'

In [68]:
reference_model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map=device,quantization_config=bnb_config,)

2025-01-26 09:43:42,866 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/config.json HTTP/11" 200 0
Loading checkpoint shards: 100%|██████████| 2/2 [00:45<00:00, 22.79s/it]
2025-01-26 09:44:28,932 https://huggingface.co:443 "HEAD /Qwen/Qwen2.5-Coder-3B-Instruct/resolve/main/generation_config.json HTTP/11" 200 0


In [69]:
from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">AFTER</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

def prompt_to_chat(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text

def infer(model, prompt, l=100, use_chat = True, temperature=0.4, top_p = 0.8):
    if use_chat:
        prompt = prompt_to_chat(prompt)
    model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=l,
        temperature=temperature, 
        top_p=top_p,
        do_sample=True ,  
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]
    return response


prompts =  [
    'Как в BlackBox можно вывести данные на консоль?', 
    'Какой тип данных в Component Pascal используется для хранения целых чисел?', 
    'МОДУЛЬ i21егэDemo2010C4ru;', 
    'Component Pascal is Oberon microsystems refinement of?', 
    'Log.String(', 
    'Типом целой константы является'
]  # feel free to add a few more that are not 100% assiciated with Python

In [70]:
def draw_diff(use_chat,l):
    print(f"USING CHAT TEMPLATE = {use_chat}")
        
    baseline = [infer(reference_model, p, l = l, use_chat=use_chat) for p in prompts]
    check = [infer(model, p, l = l, use_chat=use_chat) for p in prompts]
    rows = []
    for i, prompt in enumerate(prompts):
        # replace placeholders in the format() arguments
        rows.append(row_template.format(prompt, baseline[i], check[i]))
    display(HTML(table_template.format('\n'.join(rows))))

In [71]:
draw_diff(True, 256)

USING CHAT TEMPLATE = True


PROMPT,BEFORE,AFTER
`Как в BlackBox можно вывести данные на консоль?`,"В BlackBox (который похож на Ansible) вы можете выводить данные на консоль с помощью различных методов. Вот несколько распространенных способов: 1. **Использование `stdout` модуля:** Вы можете использовать модуль `stdout` для вывода данных на консоль. Например: ```yaml - name: Print message to stdout debug: msg: ""Hello, World!"" ``` 2. **Использование `shell` модуля:** Вы можете использовать модуль `shell` для выполнения команд и вывода их результатов на консоль. Например: ```yaml - name: Execute command and print output shell: echo ""Hello, World!"" ``` 3. **Использование `command` модуля:** Similar to `shell`, but it is generally preferred for executing commands that do not require input/output redirection. ```yaml - name: Execute command and print output command: echo ""Hello, World!"" ``` 4. **Использование `set_fact` модуля:** You can set facts and then use the `debug` module","В BlackBox Component Builder есть несколько способов вывода информации в консоль. Для этого используется модуль StdLog. В следующем примере показано, как можно использовать его для вывода текста в консоль: MODULE LogExample; IMPORT StdLog; BEGIN StdLog.String('Hello World'); (* записываем ""Hello World"" в консоль *) StdLog.Ln; (* перевод строки *) StdLog.Int(123456); (* записываем целое число 123456 в консоль *) StdLog.Real(3.14159); (* записываем вещественное число 3.14159 в консоль *) StdLog.Real(3.14159E-2); (* записываем вещественное число 0.314159 в консоль *) StdLog.Real(3.14159E+2); (* записываем вещественное число 314.159 в консоль *) StdLog.Real(3.14159E-2, 0,"
`Какой тип данных в Component Pascal используется для хранения целых чисел?`,В Component Pascal используется тип данных `Integer` для хранения целых чисел.<|im_end|>,В Component Pascal целые числа представлены типом INTEGER.<|im_end|>
`МОДУЛЬ i21егэDemo2010C4ru;`,"It looks like you've provided a module name `i21egDemo2010C4ru`. However, without more context or information about what this module is supposed to do, I can't provide a detailed explanation or help with it. If you're trying to create or use a module in a programming language like C#, Java, Python, etc., please provide more details about the module's purpose and any specific functionality you need assistance with. This will help me give you more targeted advice and support.<|im_end|>","Модуль создан 2010-09-05, последний редактированный 2010-09-05, последняя версия 1.<|im_end|>"
`Component Pascal is Oberon microsystems refinement of?`,"Component Pascal is an extension of the Oberon programming language developed by Oberon Microsystems. It was designed to provide a more structured and object-oriented approach to software development, while still retaining the simplicity and efficiency of the original Oberon language. Some key features of Component Pascal include: 1. Object-oriented programming: Component Pascal supports object-oriented concepts such as classes, inheritance, and polymorphism. 2. Components: Components are reusable building blocks that encapsulate related functionality. 3. Interfaces: Interfaces define the contract for components, allowing them to interact with each other. 4. Dynamic binding: Component Pascal uses dynamic binding to resolve method calls at runtime. Component Pascal was developed in the early 1980s and was used in various applications, including embedded systems, real-time operating systems, and multimedia applications. It has since been replaced by other languages such as C++ and Java, but it remains an important language in the history of programming.<|im_end|>",Component Pascal is a variant of the Oberon programming language.<|im_end|>
`Log.String(`,"It looks like you're trying to log a string in a programming context. However, the specific syntax and library used can vary depending on the programming language you're working with. Below are examples of how you might log a string in some common languages: ### Python In 

In [72]:
draw_diff(False, 39)

USING CHAT TEMPLATE = False


PROMPT,BEFORE,AFTER
`Как в BlackBox можно вывести данные на консоль?`,"В BlackBox, для вывода данных на консоль можно использовать команды `echo` или `print`. Вот пример использования этих команд: ```python # Использование","Как вставить в текстовые документы символы, которые не печатаются на экране (например, пробелы шириной в один пиксель)? Как вставить в"
`Какой тип данных в Component Pascal используется для хранения целых чисел?`,Какие операции можно выполнять с этим типом данных? В Component Pascal используется тип данных `Integer` для хранения целых чисел. Этот тип данных может быть использован для,"Числа, которые могут быть представлены в диапазоне от -32768 до 32767 (включительно), называются short integers"
`МОДУЛЬ i21егэDemo2010C4ru;`,// Использование модуля i21егэDemo2010C4ru // Определение функции для работы с модулем void i21,(* Задание 4. Смешанный пример с комментариями. *) ПОДКЛЮЧИТЬ Вывод := i21
`Component Pascal is Oberon microsystems refinement of?`,? A. Pascal B. C C. Java D. Python The correct answer is A. Pascal. Oberon microsystems was a company that developed a programming language called,"(a) Modula-2, (b) Pascal, (c) C, (d) Smalltalk. Answer: (a). The following statements are about the language Pascal:"
`Log.String(`,"“Error:”, err.Error()) } } func main() { // Create a new instance of the logger logger := NewLogger() // Set the log level to DEBUG logger",""" ""); Log.Int( x ); Log.Ln; END PrintTree; PROCEDURE ( v: Tree ) Insert* ( x: INTEGER ), NEW; VAR"
`Типом целой константы является`,: 1. Число с плавающей точкой 2. Целое число 3. Символ 4. Строковое значение Правильный ответ,"целый тип, т.е. INTEGER, SHORTINT, BYTE, LONGINT, SHORTREAL, REAL, CHAR, SET или BOOLEAN. Целые константы могут быть"


In [73]:
print(infer(model, 'Ты помошник по среде Component Pascal. Ответь на вопрос: Как мне вывести строку hello в окно лога?', 256, use_chat = True))

Чтобы вывести текст в окно лога, нужно использовать процедуру Log.String('текст').<|im_end|>


In [80]:
print(infer(model, 'Напиши процедуру сортировки массива.',  1024, use_chat = True))

Процедура сортировки пузырьком (Bubble Sort) - это простейшая и наивная реализация сортировки. Она состоит из повторяющихся проходов по массиву, в котором каждый раз сравниваются соседние элементы. Если они в неправильном порядке, то они меняются местами. Проходы продолжаются до тех пор, пока не будет сделано необходимое количество проходов, чтобы упорядочить весь массив.

PROCEDURE BubbleSort ( VAR arr: ARRAY OF INTEGER );
	VAR  i, j: INTEGER;  swapped: BOOLEAN;
BEGIN
	swapped := TRUE;
	WHILE swapped DO
		swapped := FALSE;
		FOR i := 0 TO LEN(arr) - 2 DO
			j := i + 1;
			IF arr[i] > arr[j] THEN
				arr[i] := arr[j];  arr[j] := arr[i];
				swapped := TRUE
			END
		END
	END
END BubbleSort;

После выполнения процедуры массив arr будет отсортирован по возрастанию.<|im_end|>


In [82]:
print(infer(model, 'перепиши код с python на component pascal. Код: def get_vowels(String): return [each for each in String if each in "aeiou"]',  1024, use_chat = True))

В Component Pascal нет такого выражения, как в Python. Однако можно написать следующий код:

MODULE  i21примПарсинг;
	IMPORT  Log := StdLog,  In := i21sysIn;
	
	PROCEDURE GetVowels* ( IN str: ARRAY OF CHAR;  OUT vowels: ARRAY OF CHAR );
		VAR  i, j: INTEGER;
	BEGIN
		i := 0;  j := 0;
		WHILE  (i < LEN(str)) & (j < LEN(vowels))  DO
			IF  "AEIOUaeiou" [ORD(str[i])] # 0X  THEN
				vowels[j] := str[i];
				INC(j)
			END;
			INC(i)
		END;
	END GetVowels;

BEGIN
	(* тест *)
		GetVowels('Это пример строки для проверки.', v);
		Log.String(v);  Log.Ln;
		GetVowels('Конечно.', v);
		Log.String(v);  Log.Ln;
	END i21примПарсинг.

Чтобы выполнить программу, нужно кликнуть по коммандеру:
DevLinker.Link i21примПарсинг.i21примПарсинг<|im_end|>


In [8]:
checkpoint = transformers.AutoModelForCausalLM.from_pretrained("./results/20250122-220441/checkpoint-1220/", device_map=device)

In [14]:
print(infer(checkpoint, 'Какой тип использовать для хранения целых чисел?', 256, use_chat = True))

Для целых чисел обычно используют тип INTEGER. Однако, если значения могут быть только некоторым предопределенным количеством значений, следует использовать тип SHORTINT, чтобы избежать возможных потерь точности, которые могут возникнуть при делении на 2. В некоторых случаях лучше использовать LONGINT.

Все последующие примеры использует LONGINT в качестве основного типа. Если результаты вычислений становятся непредсказуемыми, если используется другой тип.

Пример:
	
	PROCEDURE Ddouble* (a, b: LONGINT);
	BEGIN
		DIAGNOSTIC;
		StdLog.Nl;
		StdLog.String("Ddouble "); StdLog.Int(a + b, 0); StdLog.Ln;
		StdLog.String(DefaCT(a + b)); StdLog.Ln
	END Ddouble;

В этом примере использовано значение эффекта (defacto) стандартного модуля Math, который включает в себя итоговое значение.
После применения команд DIAGNOSTIC (см. ниже), он будет выровнен по левому краю, а не


In [ ]:
def format_chat(q,a):
    # Create a chat-like format using the tokenizer's chat template
    chat = [
        {"role": "user", "content": q},
        {"role": "assistant", "content": a}
    ]
    # Apply the chat template
    return tokenizer.apply_chat_template(chat, tokenize=False)




In [58]:
chat_str = format_chat("вопрос", "ответ").strip()
print(chat_str + tokenizer.eos_token)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
вопрос<|im_end|>
<|im_start|>assistant
ответ<|im_end|><|im_end|>


In [65]:
print(tokenizer(chat_str, add_special_tokens = True, max_length=128,  padding='max_length'))

{'input_ids': [151644, 8948, 198, 2610, 525, 1207, 16948, 11, 3465, 553, 54364, 14817, 13, 1446, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 5474, 28156, 40957, 151645, 198, 151644, 77091, 198, 127227, 151645, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643], 'attention_mask': [1, 

In [ ]:






print("ends fith eos", tokenizer(chat_str, add_special_tokens = True)["input_ids"][-1]== tokenizer.eos_token_id)
print("has attention", tokenizer(chat_str)["attention_mask"])



chat_str = chat_str + tokenizer.eos_token + tokenizer.pad_token
print("has attention2", tokenizer(chat_str, add_special_tokens =tokenizer.pad_token_id)["attention_mask"])

In [48]:
tokenizer.pad_token, tokenizer.pad_token_id

('<|endoftext|>', 151643)

In [49]:
tokenizer.eos_token, tokenizer.eos_token_id

('<|im_end|>', 151645)

In [ ]:
model config
"bos_token_id": 151643,
"eos_token_id": 151645,
tokenizer config
"eos_token": "<|im_end|>",
  "errors": "replace",
  "model_max_length": 32768,
  "pad_token": "<|endoftext|>",